In [6]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.misc as ms
import fitter.model_average as md
#import fitter.fitter as fit

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [10]:
from importlib import reload
reload(md)
reload(dl)
reload(ms)

<module 'fitter.misc' from '../fitter/misc.py'>

In [11]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_results('test')
#other_results = data_loader.get_fit_results('optimized_priors_a4')
other_results = data_loader.get_fit_results('other_collabs')

model_average = md.model_average(fit_results)
print(model_average)
#model_average.plot_histogram('FK/Fpi', vary_choice='include_latt_n3lo')

FK/Fpi_pm = 0.898(55) 	 [FLAG: 1.1932(19)] 

---
FK/Fpi      =  0.896(55) 
delta_su(2) = 0.0042(17) 

---
Uncertainty: 
   Unexplained: 0.055 
   Explained:   0.0 

---
Error Budget: 
   Chiral:      0.024010443573120783 
   Phys Point:  0.036492153233449315 
   Statistical: 0.03355753921521225 



In [12]:
for model in model_average.get_model_names():
    fit_info = model_average.fit_results[model]
    prior = model_average._get_fit_prior(model)
    print(model, fit_info['Q'],  fit_info['FK/Fpi'])
    for key in ['A_a', 'A_loga', 'A_aa']:
        if key in prior:
            print(key, prior[key])
    print('--')

poly_FpiFpi_nnlo_bijnens 9.775841134451406e-108 1.290(46)


TypeError: argument of type 'NoneType' is not iterable

### Make plots

In [5]:
model_average.plot_fits('a');

In [13]:
model_average.plot_histogram('FK/Fpi_pm', vary_choice='F2');

In [ ]:
model_average.average('FK')

In [15]:
model_average.plot_comparison(other_results=other_results, param='FK/Fpi_pm', show_model_avg=True);

In [16]:
model_average.plot_comparison(other_results=other_results, param='FK/Fpi_pm', show_model_avg=True);